# Data Quality Analysis - Brazilian E-Commerce Dataset

This notebook focuses on identifying and documenting data quality issues and missing values across all datasets.

## Objectives
1. Identify missing values in each dataset
2. Analyze data types and inconsistencies
3. Detect duplicates and outliers
4. Document data quality issues
5. Create recommendations for data cleaning

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

## 1. Load All Datasets

In [ ]:
# Define paths
RAW_DATA_PATH = '../data/raw/'
REPORTS_PATH = '../reports/'

# Ensure reports directory exists
os.makedirs(REPORTS_PATH, exist_ok=True)

# Define dataset files
datasets = {
    'orders': 'olist_orders_dataset.csv',
    'customers': 'olist_customers_dataset.csv',
    'order_items': 'olist_order_items_dataset.csv',
    'order_payments': 'olist_order_payments_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'sellers': 'olist_sellers_dataset.csv',
    'geolocation': 'olist_geolocation_dataset.csv',
    'category_translation': 'product_category_name_translation.csv'
}

# Load all datasets
data = {}

for name, filename in datasets.items():
    filepath = os.path.join(RAW_DATA_PATH, filename)
    try:
        df = pd.read_csv(filepath)
        data[name] = df
        print(f"Loaded {name}: {df.shape[0]:,} rows x {df.shape[1]} columns")
    except Exception as e:
        print(f"Error loading {name}: {e}")

print(f"\nTotal datasets loaded: {len(data)}")

## 2. Comprehensive Data Quality Report

In [ ]:
# Function to analyze data quality for a single dataset
def analyze_data_quality(df, dataset_name):
    """
    Analyze data quality for a dataset.
    
    Returns a dictionary with quality metrics.
    """
    quality_report = {
        'dataset': dataset_name,
        'total_rows': df.shape[0],
        'total_columns': df.shape[1],
        'total_cells': df.shape[0] * df.shape[1],
        'duplicate_rows': df.duplicated().sum(),
        'duplicate_percentage': round((df.duplicated().sum() / df.shape[0]) * 100, 2),
        'memory_usage_mb': round(df.memory_usage(deep=True).sum() / 1024**2, 2)
    }
    
    # Analyze missing values
    missing_info = []
    for col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            missing_pct = round((missing_count / df.shape[0]) * 100, 2)
            missing_info.append({
                'column': col,
                'missing_count': missing_count,
                'missing_percentage': missing_pct,
                'data_type': str(df[col].dtype)
            })
    
    quality_report['columns_with_missing'] = len(missing_info)
    quality_report['missing_details'] = missing_info
    
    # Analyze data types
    dtype_info = df.dtypes.value_counts().to_dict()
    quality_report['data_types'] = dtype_info
    
    return quality_report

# Analyze all datasets
all_quality_reports = {}

for name, df in data.items():
    print(f"\n{'='*60}")
    print(f"Analyzing: {name.upper()}")
    print(f"{'='*60}")
    
    report = analyze_data_quality(df, name)
    all_quality_reports[name] = report
    
    # Print summary
    print(f"Rows: {report['total_rows']:,}")
    print(f"Columns: {report['total_columns']}")
    print(f"Duplicate rows: {report['duplicate_rows']:,} ({report['duplicate_percentage']}%)")
    print(f"Columns with missing values: {report['columns_with_missing']}")
    print(f"Memory usage: {report['memory_usage_mb']} MB")
    
    # Print missing value details
    if report['missing_details']:
        print(f"\nMissing Values:")
        for item in report['missing_details']:
            print(f"  - {item['column']}: {item['missing_count']:,} ({item['missing_percentage']}%)")

## 3. Detailed Missing Value Analysis

In [ ]:
# Create comprehensive missing value report
missing_report = []

for name, df in data.items():
    for col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            missing_pct = round((missing_count / df.shape[0]) * 100, 2)
            
            # Determine severity
            if missing_pct > 50:
                severity = 'CRITICAL'
            elif missing_pct > 20:
                severity = 'HIGH'
            elif missing_pct > 5:
                severity = 'MEDIUM'
            else:
                severity = 'LOW'
            
            missing_report.append({
                'Dataset': name,
                'Column': col,
                'Missing_Count': missing_count,
                'Missing_Percentage': missing_pct,
                'Severity': severity,
                'Data_Type': str(df[col].dtype),
                'Total_Rows': df.shape[0]
            })

# Create DataFrame for missing values
missing_df = pd.DataFrame(missing_report)

if not missing_df.empty:
    # Sort by severity and percentage
    severity_order = {'CRITICAL': 0, 'HIGH': 1, 'MEDIUM': 2, 'LOW': 3}
    missing_df['Severity_Order'] = missing_df['Severity'].map(severity_order)
    missing_df = missing_df.sort_values(['Severity_Order', 'Missing_Percentage'], ascending=[True, False])
    missing_df = missing_df.drop('Severity_Order', axis=1)
    
    print("\n" + "="*80)
    print("COMPREHENSIVE MISSING VALUE REPORT")
    print("="*80)
    display(missing_df)
    
    # Save to CSV
    missing_df.to_csv(os.path.join(REPORTS_PATH, 'missing_values_report.csv'), index=False)
    print(f"\nMissing values report saved to: {os.path.join(REPORTS_PATH, 'missing_values_report.csv')}")
else:
    print("\nNo missing values found in any dataset!")

## 4. Data Type Analysis and Inconsistencies

In [ ]:
# Analyze data types across all datasets
dtype_report = []

for name, df in data.items():
    for col in df.columns:
        dtype_report.append({
            'Dataset': name,
            'Column': col,
            'Data_Type': str(df[col].dtype),
            'Non_Null_Count': df[col].count(),
            'Null_Count': df[col].isnull().sum(),
            'Unique_Values': df[col].nunique()
        })

dtype_df = pd.DataFrame(dtype_report)

print("\n" + "="*80)
print("DATA TYPE ANALYSIS")
print("="*80)
display(dtype_df)

# Save to CSV
dtype_df.to_csv(os.path.join(REPORTS_PATH, 'data_types_report.csv'), index=False)
print(f"\nData types report saved to: {os.path.join(REPORTS_PATH, 'data_types_report.csv')}")

## 5. Date Column Analysis

In [ ]:
# Identify and analyze date columns
date_columns = []

for name, df in data.items():
    for col in df.columns:
        # Check if column name suggests it's a date
        if any(keyword in col.lower() for keyword in ['date', 'time', 'timestamp']):
            date_columns.append({
                'Dataset': name,
                'Column': col,
                'Current_Type': str(df[col].dtype),
                'Sample_Values': df[col].dropna().head(3).tolist()
            })

if date_columns:
    date_df = pd.DataFrame(date_columns)
    print("\n" + "="*80)
    print("DATE COLUMNS IDENTIFIED")
    print("="*80)
    display(date_df)
    
    # Save to CSV
    date_df.to_csv(os.path.join(REPORTS_PATH, 'date_columns_report.csv'), index=False)
    print(f"\nDate columns report saved to: {os.path.join(REPORTS_PATH, 'date_columns_report.csv')}")
else:
    print("\nNo date columns identified.")

## 6. Duplicate Analysis

In [ ]:
# Analyze duplicates in detail
duplicate_report = []

for name, df in data.items():
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        duplicate_report.append({
            'Dataset': name,
            'Duplicate_Rows': dup_count,
            'Duplicate_Percentage': round((dup_count / df.shape[0]) * 100, 2),
            'Total_Rows': df.shape[0]
        })
        
        # Show sample duplicates
        print(f"\n{'='*60}")
        print(f"Duplicates in {name}:")
        print(f"{'='*60}")
        print(f"Total duplicates: {dup_count:,} ({round((dup_count / df.shape[0]) * 100, 2)}%)")
        print(f"\nSample duplicate rows:")
        display(df[df.duplicated(keep=False)].head(6))

if duplicate_report:
    dup_df = pd.DataFrame(duplicate_report)
    dup_df.to_csv(os.path.join(REPORTS_PATH, 'duplicates_report.csv'), index=False)
    print(f"\nDuplicates report saved to: {os.path.join(REPORTS_PATH, 'duplicates_report.csv')}")
else:
    print("\nNo duplicates found in any dataset!")

## 7. Outlier Detection (Numerical Columns)

In [ ]:
# Detect outliers in numerical columns using IQR method
outlier_report = []

for name, df in data.items():
    # Select numerical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        if df[col].notna().sum() > 0:  # Only if column has non-null values
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
            outlier_count = len(outliers)
            outlier_pct = round((outlier_count / df[col].notna().sum()) * 100, 2)
            
            if outlier_count > 0:
                outlier_report.append({
                    'Dataset': name,
                    'Column': col,
                    'Outlier_Count': outlier_count,
                    'Outlier_Percentage': outlier_pct,
                    'Lower_Bound': round(lower_bound, 2),
                    'Upper_Bound': round(upper_bound, 2),
                    'Min_Value': round(df[col].min(), 2),
                    'Max_Value': round(df[col].max(), 2)
                })

if outlier_report:
    outlier_df = pd.DataFrame(outlier_report)
    print("\n" + "="*80)
    print("OUTLIER DETECTION REPORT (IQR Method)")
    print("="*80)
    display(outlier_df)
    
    # Save to CSV
    outlier_df.to_csv(os.path.join(REPORTS_PATH, 'outliers_report.csv'), index=False)
    print(f"\nOutliers report saved to: {os.path.join(REPORTS_PATH, 'outliers_report.csv')}")
else:
    print("\nNo outliers detected in numerical columns.")

## 8. Data Quality Summary Dashboard

In [ ]:
# Create comprehensive data quality summary
quality_summary = []

for name, df in data.items():
    total_cells = df.shape[0] * df.shape[1]
    missing_cells = df.isnull().sum().sum()
    missing_pct = round((missing_cells / total_cells) * 100, 2)
    
    # Count columns with missing values
    cols_with_missing = sum(df[col].isnull().sum() > 0 for col in df.columns)
    
    # Count numerical columns
    numeric_cols = len(df.select_dtypes(include=[np.number]).columns)
    
    # Count categorical columns
    categorical_cols = len(df.select_dtypes(include=['object']).columns)
    
    quality_summary.append({
        'Dataset': name,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Numeric_Columns': numeric_cols,
        'Categorical_Columns': categorical_cols,
        'Missing_Cells': missing_cells,
        'Missing_Percentage': missing_pct,
        'Columns_With_Missing': cols_with_missing,
        'Duplicate_Rows': df.duplicated().sum(),
        'Memory_MB': round(df.memory_usage(deep=True).sum() / 1024**2, 2)
    })

summary_df = pd.DataFrame(quality_summary)
summary_df = summary_df.sort_values('Missing_Percentage', ascending=False)

print("\n" + "="*100)
print("DATA QUALITY SUMMARY DASHBOARD")
print("="*100)
display(summary_df)

# Save to CSV
summary_df.to_csv(os.path.join(REPORTS_PATH, 'data_quality_summary.csv'), index=False)
print(f"\nData quality summary saved to: {os.path.join(REPORTS_PATH, 'data_quality_summary.csv')}")

## 9. Recommendations for Data Cleaning

In [ ]:
# Generate recommendations based on data quality analysis
recommendations = []

for name, df in data.items():
    # Check for missing values
    for col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            missing_pct = round((missing_count / df.shape[0]) * 100, 2)
            
            if missing_pct > 50:
                action = "DROP COLUMN - Too many missing values"
                priority = "HIGH"
            elif missing_pct > 20:
                action = "IMPUTE or DROP ROWS - Consider imputation or removal"
                priority = "MEDIUM"
            elif missing_pct < 5:
                action = "IMPUTE - Use mean/median/mode or forward fill"
                priority = "LOW"
            else:
                action = "REVIEW - Investigate cause of missing values"
                priority = "MEDIUM"
            
            recommendations.append({
                'Dataset': name,
                'Column': col,
                'Missing_Percentage': missing_pct,
                'Priority': priority,
                'Recommended_Action': action
            })
    
    # Check for duplicates
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        recommendations.append({
            'Dataset': name,
            'Column': 'ALL_COLUMNS',
            'Missing_Percentage': 0,
            'Priority': 'MEDIUM',
            'Recommended_Action': f'REMOVE {dup_count} DUPLICATE ROWS'
        })

if recommendations:
    rec_df = pd.DataFrame(recommendations)
    
    # Sort by priority
    priority_order = {'HIGH': 0, 'MEDIUM': 1, 'LOW': 2}
    rec_df['Priority_Order'] = rec_df['Priority'].map(priority_order)
    rec_df = rec_df.sort_values(['Priority_Order', 'Missing_Percentage'], ascending=[True, False])
    rec_df = rec_df.drop('Priority_Order', axis=1)
    
    print("\n" + "="*100)
    print("DATA CLEANING RECOMMENDATIONS")
    print("="*100)
    display(rec_df)
    
    # Save to CSV
    rec_df.to_csv(os.path.join(REPORTS_PATH, 'cleaning_recommendations.csv'), index=False)
    print(f"\nCleaning recommendations saved to: {os.path.join(REPORTS_PATH, 'cleaning_recommendations.csv')}")
else:
    print("\nNo data quality issues found. Data is clean!")

## 10. Summary and Next Steps

In [ ]:
print("="*100)
print("DATA QUALITY ANALYSIS COMPLETE")
print("="*100)

print("\nGenerated Reports:")
print("  1. missing_values_report.csv - Detailed missing value analysis")
print("  2. data_types_report.csv - Data type information")
print("  3. date_columns_report.csv - Date column identification")
print("  4. duplicates_report.csv - Duplicate row analysis")
print("  5. outliers_report.csv - Outlier detection")
print("  6. data_quality_summary.csv - Overall quality dashboard")
print("  7. cleaning_recommendations.csv - Actionable recommendations")

print("\nNext Steps:")
print("  1. Review the cleaning recommendations")
print("  2. Prioritize HIGH priority issues")
print("  3. Create data cleaning scripts")
print("  4. Validate cleaned data")
print("  5. Proceed with advanced analysis")

print("\nKey Findings:")
print(f"  - Total datasets analyzed: {len(data)}")
print(f"  - Total rows across all datasets: {sum(df.shape[0] for df in data.values()):,}")
print(f"  - Datasets with missing values: {sum(1 for df in data.values() if df.isnull().sum().sum() > 0)}")
print(f"  - Datasets with duplicates: {sum(1 for df in data.values() if df.duplicated().sum() > 0)}")